In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Define the system of equations with time-dependent alpha
def sirs_time_dependent_alpha(t, y, beta, gamma, alpha_function, delta):
    S, I, R = y
    alpha = alpha_function(t)
    dSdt = -beta*I*S + alpha*R - delta*S
    dIdt = beta*I*S - gamma*I
    dRdt = gamma*I - alpha*R + delta*S
    return [dSdt, dIdt, dRdt]

# Define a time-dependent alpha function (e.g., linear increase over time)
def alpha_linear_increase(t):
    return 0.15

# Set the initial conditions and parameters
S0 = 0.99
I0 = 0.01
R0 = 0
beta = 0.3
gamma = 0.1
delta = 0.00
t_span = (0, 1000)

# Solve the system of equations numerically with time-dependent alpha
sol = solve_ivp(sirs_time_dependent_alpha, t_span, [S0, I0, R0], args=(beta, gamma, alpha_linear_increase, delta))

# Plot the results
plt.plot(sol.t, sol.y[0], label='S')
plt.plot(sol.t, sol.y[1], label='I')
plt.plot(sol.t, sol.y[2], label='R')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Fraction of population')
plt.show()


In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Define the SIRS model
def SIRS(y, t, beta, gamma, nu):
    S, I, R = y
    dSdt = -beta * S * I + nu * R
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I - nu * R
    return [dSdt, dIdt, dRdt]

# Define the parameters
N = 1000 # total population size
I0 = 1 # initial number of infected individuals
R0 = 0 # initial number of immune individuals
S0 = N - I0 - R0 # initial number of susceptible individuals
beta = 0.005 # transmission rate
gamma = 0.04 # recovery rate
nu = 0.05 # loss of immunity rate
t = np.linspace(0, 1000, 1000) # time points to simulate over

# Solve the ODE system using odeint
sol = odeint(SIRS, [S0, I0, R0], t, args=(beta, gamma, nu))

# Plot the results
plt.plot(t, sol[:, 0], label='Susceptible')
plt.plot(t, sol[:, 1], label='Infected')
plt.plot(t, sol[:, 2], label='Recovered')
plt.xlabel('Time')
plt.ylabel('Number of individuals')
plt.legend()
plt.show()


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Parameters
beta = 0.5
gamma = 0.1
mu = 0.05
N = 1000
t_max = 500

# Time vector
t = np.linspace(0, t_max, t_max + 1)

# Initial conditions
S0, I0, R0 = N-1, 1, 0
y0 = [S0, I0, R0]

# Function to calculate the loss of immunity rate at time t
def loss_of_immunity(t):
    return 0.05 + 0.01 * np.sin(2*np.pi*t/365)

# SIRS model without convolution term
def SIRS(t, y):
    S, I, R = y
    dS_dt = -beta*S*I/N + mu*R #+ loss_of_immunity(t)*R
    dI_dt = beta*S*I/N - loss_of_immunity(t)*I
    dR_dt = loss_of_immunity(t)*I - mu*R #- loss_of_immunity(t)*R
    return [dS_dt, dI_dt, dR_dt]

# Solve the differential equation
sol = solve_ivp(SIRS, [0, t_max], y0, t_eval=t)

# Plot the results
plt.plot(sol.t, sol.y[0], label='S')
plt.plot(sol.t, sol.y[1], label='I')
plt.plot(sol.t, sol.y[2], label='R')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('SIRS Model with Time-Dependent Loss of Immunity')
plt.legend()
plt.show()


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters
#beta = 0.25*0 + 0.5
#gamma = 1/15*0 + 0.1
#mu = 0.097*0 + 0.1455
#N = 10000
#t_max = 500
#eta = 1.5
#zeta = 0.9995
#ls = eta*beta
#g0 = zeta*gamma
#a0 = 0.0007
#mu1 = (zeta*0.8)*mu

beta = 0.25*0 + 0.5
gamma = 1/15*0 + 0.1
mu = 0.097*0 + 0.13#0.1455
N = 10000
t_max = 1500
eta = 1.5
zeta = 0.9995
ls = eta*beta
g0 = zeta*gamma
a0 = 0.0001
mu1 = (zeta*0.5)*mu

entry = int(t_max/3)
t_maxfast = t_max - entry

# Time vector
t = np.linspace(0, t_max, t_max + 1)
tlist = list(t)
t1 = np.linspace(0, t_maxfast, t_maxfast + 1)

# Initial conditions
S0, I0, R0 = N-1, 1, 0
y0 = [S0, I0, R0]

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

g = lambda t : array([S0,I0,R0])


d = 10
yy = ddeint(model,g,t1,fargs=(d,))

def gammanew(t):
    if t < entry:
        return g0
    else:
        print(t-entry)
        return g0*(1 - a0*yy[t-entry][1])

def modelslow(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    #t_index = tlist.index(t)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])
f = lambda t : array([S0,I0,R0])
d1 = 10
xx = ddeint(modelslow,f,t,fargs=(d1,))
add = np.zeros(entry)
plot = [row[1] for row in yy]
plot = np.array(plot)
plot1 = np.concatenate((add,plot))
#print(plot1)

# Plot the results
#plt.plot(t, yy[:,0], label='S')
#plt.plot(t1, yy[:,1], label='I')
#plt.plot(t, yy[:,2], label='R')
plt.plot(t, xx[:,1], label='I')
plt.plot(t, plot1, label='Ifast')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('SIRS Model with Time-Dependent Gamma')
plt.legend()
plt.show()


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters
#beta = 0.25*0 + 0.5
#gamma = 1/15*0 + 0.1
#mu = 0.097*0 + 0.1455
#N = 10000
#t_max = 500
#eta = 1.8
#zeta = 0.7
#ls = eta*beta
#g0 = zeta*gamma
#a0 = 0.0007
#mu1 = zeta*mu

beta = 0.25*0 + 0.5
gamma = 1/15*0 + 0.1
mu = 0.097*0 + 0.13#0.1455
N = 10000
t_max = 1500
eta = 1.5
zeta = 0.9995
ls = eta*beta
g0 = zeta*gamma
a0 = 0.0001
mu1 = (zeta*0.5)*mu

entry = int(t_max/3)
t_maxfast = t_max - entry

# Time vector
t = np.linspace(0, t_max, t_max + 1)
tlist = list(t)
t1 = np.linspace(0, t_maxfast, t_maxfast + 1)

# Initial conditions
S0, I0, R0 = N-1, 1, 0
y0 = [S0, I0, R0]

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

g = lambda t : array([S0,I0,R0])


d = 10
yy = ddeint(model,g,t1,fargs=(d,))

add = np.zeros(entry)
add1 = np.zeros(entry)#+10)
plot = [row[1] for row in yy]
plot = np.array(plot)
plot1 = np.concatenate((add,plot))
plot2 = np.concatenate((add1,plot))

def gammanew(t):
    if t < entry:
        return g0
    else:
        return max(0, g0*(1 - a0*plot2[int(t)]))

def modelslow(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - gammanew(t)*I, gammanew(t)*I - mu1*Rd])
f = lambda t : array([S0,I0,R0])
d1 = 5
xx = ddeint(modelslow,f,t,fargs=(d1,))

#print(plot1)

# Plot the results
#plt.plot(t, yy[:,0], label='S')
#plt.plot(t1, yy[:,1], label='I')
#plt.plot(t, yy[:,2], label='R')
plt.plot(t, xx[:,1], label='I')
plt.plot(t, plot1, label='Ifast')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('SIRS Model with Time-Dependent Gamma')
plt.legend()
plt.show()


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)
beta = 0.25*0 + 0.5
gamma = 1/15*0 + 0.1
mu = 0.097*0 + 0.13#0.1455
N = 10000
t_max = 1500
eta = 1.5
zeta = 0.9995
ls = eta*beta
g0 = zeta*gamma
a0 = 0.0001
mu1 = (zeta*0.5)*mu

entry = 165
t_maxfast = t_max - entry

# Time vector
t = np.linspace(0, t_max, t_max + 1)
tlist = list(t)
t1 = np.linspace(0, t_maxfast, t_maxfast + 1)

# Initial conditions
S0, I0, R0 = N-1, 1, 0
y0 = [S0, I0, R0]

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])

g = lambda t : array([S0,I0,R0])


d = 10
yy = ddeint(model,g,t1,fargs=(d,))

add = np.zeros(entry)
add1 = np.zeros(entry+10)
plot = [row[1] for row in yy]
plot = np.array(plot)
plot1 = np.concatenate((add,plot))
plot2 = np.concatenate((add1,plot))



def gammanew(t):
    if t < entry:
        return g0
    else:
        return max(0, gamma*(1 - a0*plot2[int(t)]))

def modelslow(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t)*I, gammanew(t)*I - mu*Rd])
f = lambda t : array([S0,I0,R0])
d1 = 10
xx = ddeint(modelslow,f,t,fargs=(d1,))

#print(plot1)

# Plot the results
#plt.plot(t, yy[:,0], label='S')
#plt.plot(t1, yy[:,1], label='I')
#plt.plot(t, yy[:,2], label='R')
plt.plot(t, xx[:,1], label='I')
plt.plot(t, plot1, label='Ifast')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('SIRS Model with Time-Dependent Gamma')
plt.legend()
plt.show()


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)
beta = 0.25*0 + 0.5
gamma = 1/15*0 + 0.1
mu = 0.097*0 + 0.13#0.1455
N = 10000
t_max = 1500
eta = 1.5
zeta = 0.9995
ls = eta*beta
g0 = zeta*gamma
a0 = 0.0001
mu1 = (zeta*0.8)*mu

entry = 165
t_maxfast = t_max - entry

# Time vector
t = np.linspace(0, t_max, t_max + 1)
tlist = list(t)
t1 = np.linspace(0, t_maxfast, t_maxfast + 1)

# Initial conditions
S0, I0, R0 = N-1, 1, 0
y0 = [S0, I0, R0]

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])

g = lambda t : array([S0,I0,R0])


d = 10
yy = ddeint(model,g,t1,fargs=(d,))

add = np.zeros(entry)
add1 = np.zeros(entry+10)
plot = [row[1] for row in yy]
plot = np.array(plot)
plot1 = np.concatenate((add,plot))
plot2 = np.concatenate((add1,plot))



def gammanew(t):
    if t < entry:
        return g0
    else:
        return max(0, gamma*(1 - a0*plot2[int(t)]))

def modelslow(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t)*I, gammanew(t)*I - mu*Rd])
f = lambda t : array([S0,I0,R0])
d1 = 10
xx = ddeint(modelslow,f,t,fargs=(d1,))

#print(plot1)

# Plot the results
#plt.plot(t, yy[:,0], label='S')
#plt.plot(t1, yy[:,1], label='I')
#plt.plot(t, yy[:,2], label='R')
plt.plot(t, xx[:,1], label='I')
plt.plot(t, plot1, label='Ifast')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('SIRS Model with Time-Dependent Gamma')
plt.legend()
plt.show()


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return g0
    else:
        return max(0, gamma*(1 - a0*plot2[int(t)]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def main(val1,val2,val3,val4,val5):
    global entry
    entry = val1
    global eta
    eta = val2
    global zeta
    zeta = val3
    global factor
    factor = val4
    
    global beta
    beta = 0.25*0 + 0.5
    global gamma
    gamma = 1/15*0 + 0.1
    global mu
    mu = val5#0.097*0 + 0.13#0.1455
    global N
    N = 10000
    t_max = 1500
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = 0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    
    plt.plot(t, xx[:,1], label='I')
    plt.plot(t, plot1, label='Ifast')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    plt.legend()
    plt.show()
    return
for i in range(5):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    main(165+50*i,1.4+0.2*j,1.2+0.0495*k,0.7+0.1*l,0.11+0.01*m)

In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return gamma
    else:
        return max(0, gamma*(1 - a0*plot2[int(t)]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def main(val1,val2,val3,val4,val5):
    global entry
    entry = val1
    global eta
    eta = val2
    global zeta
    zeta = val3
    global factor
    factor = val4
    
    global beta
    beta = 0.25*0 + 0.5
    global gamma
    gamma = 1/15*0 + 0.1
    global mu
    mu = val5#0.097*0 + 0.13#0.1455
    global N
    N = 10000
    t_max = 1500
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = 0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    
    plt.plot(t, xx[:,1], label='Epidemic')
    plt.plot(t, plot1, label='Pandemic')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    plt.legend()
    plt.show()
    return

main(165,1.4,1.2,0.9,0.11)
main(165,1.4,1.2495,0.9,0.13)
main(165,1.4,1.3,0.9,0.13)
main(215,1.4,1.3,0.7,0.13)
main(215,1.8,1.3,0.9,0.11)
main(265,1.4,1.2,0.7,0.11)
main(265,1.4,1.2,0.9,0.13)
main(265,1.4,1.2495,0.9,0.13)
main(600,1.6,1.3,0.7,0.11)
main(365,1.6,1.3,0.7,0.12)

In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return gamma
    else:
        return max(0, gamma*(1 - a0*plot2[int(floor(t))]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def modelslownormal(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def stationaryval(values):
    count = 0
    prev = values[0]
    for i in range(len(values)-1):
        if count == 100:
            return prev
        if values[i+1] - prev <= 1:
            count += 1
        else:
            count = 0
        prev = values[i+1]
    return 0

stationary = []
changed = []
entrypoint = []

def main(val1,val2,val3,val4,val5):
    global entry
    entry = val1
    global eta
    eta = val2
    global zeta
    zeta = val3
    global factor
    factor = val4
    
    global beta
    beta = 0.25*0 + 0.5
    global gamma
    gamma = 1/15*0 + 0.1
    global mu
    mu = val5#0.097*0 + 0.13#0.1455
    global N
    N = 10000
    t_max = 1500
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = 0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]
    
    global stationary
    global changed
    global entrypoint

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    zz = ddeint(modelslownormal,f,t,fargs=(d1,))
    
    entrypoint.append(val2)
    stationary.append(stationaryval([row[1] for row in zz]))
    changed.append(stationaryval([row[1] for row in xx]))
    
    plt.plot(t, xx[:,1], label='I')
    plt.plot(t, zz[:,1], label='Inormal')
    plt.plot(t, plot1, label='Ifast')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    plt.legend()
    plt.show()
    return


main(350,1.0,1.2,0.9,0.11)
main(350,1.1,1.2,0.9,0.11)
main(350,1.2,1.2,0.9,0.11)
main(350,1.3,1.2,0.9,0.11)
main(350,1.4,1.2,0.9,0.11)
main(350,1.5,1.2,0.9,0.11)
main(350,1.6,1.2,0.9,0.11)
main(350,1.7,1.2,0.9,0.11)
main(350,1.8,1.2,0.9,0.11)

plt.plot(entrypoint, stationary, label='Without Pandemic')
plt.plot(entrypoint, changed, label='With pandemic')
plt.xlabel('Eta')
plt.ylabel('Stationary values')
plt.title('Eta vs Change in Stationary values')
plt.legend()
plt.show()

In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return gamma
    else:
        return max(0, gamma*(1 - a0*plot2[int(floor(t))]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def modelslownormal(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def stationaryval(values):
    count = 0
    prev = values[0]
    for i in range(len(values)-1):
        if count == 100:
            return prev
        if values[i+1] - prev <= 1:
            count += 1
        else:
            count = 0
        prev = values[i+1]
    return 0

stationary = []
changed = []
entrypoint = []

def main(val1,val2,val3,val4,val5):
    global entry
    entry = val1
    global eta
    eta = 1.4#val2
    global zeta
    zeta = val3
    global factor
    factor = val4
    
    global beta
    beta = val2#0.25*0 + 0.5
    global gamma
    gamma = 1/15*0 + 0.1
    global mu
    mu = 0.097*0 + 0.11#0.1455
    global N
    N = 10000
    t_max = 1500
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = val5#0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]
    
    global stationary
    global changed
    global entrypoint

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    zz = ddeint(modelslownormal,f,t,fargs=(d1,))
    
    entrypoint.append(val5)
    stationary.append(stationaryval([row[1] for row in zz]))
    changed.append(stationaryval([row[1] for row in xx]))
    
    #plt.plot(t, xx[:,1], label='I')
    #plt.plot(t, zz[:,1], label='Inormal')
    #plt.plot(t, plot1, label='Ifast')
    #plt.xlabel('Time')
    #plt.ylabel('Population')
    #plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    #plt.legend()
    #plt.show()
    return




def run(a,b,c,d,e):
    for i in range(10):
        main(a,b,c,d,e+i*0.00002)
    plt.scatter(entrypoint, stationary, label='Without Pandemic')
    plt.scatter(entrypoint, changed, label='With pandemic')
    plt.xlabel('a0')
    plt.ylabel('Stationary values')
    plt.title('a0 vs Change in Stationary values')
    plt.legend()
    plt.show()
    return

run(350,0.65,1.2,0.9,0.00006)


In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint
from scipy.integrate import trapz
# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return gamma
    else:
        return max(0, gamma*(1 - a0*plot2[int(floor(t))]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def modelslownormal(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def stationaryval(values):
    count = 0
    prev = values[0]
    for i in range(len(values)-1):
        if count == 100:
            return prev
        if values[i+1] - prev <= 1:
            count += 1
        else:
            count = 0
        prev = values[i+1]
    return 0

stationary = []
changed = []
entrypoint = []

def main(marker,val1,val2,val3,val4,val5):
    marker = marker
    global entry
    entry = val1
    global eta
    eta = 1.4#val2
    global zeta
    zeta = 1.2#val3
    global factor
    factor = val4
    
    global beta
    beta = val2#0.25*0 + 0.5
    global gamma
    gamma = val3#1/15*0 + 0.1
    global mu
    mu = 0.097*0 + 0.11#0.1455
    global N
    N = 10000
    t_max = 1500 + val1
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = val5#0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]
    
    global stationary
    global changed
    global entrypoint

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    zz = ddeint(modelslownormal,f,t,fargs=(d1,))
    
    if marker == 1:
        entrypoint.append(val1)
    elif marker == 2:
        entrypoint.append(val2)
    elif marker == 3:
        entrypoint.append(val3)
    elif marker == 4:
        entrypoint.append(val5)
    
    intval = [row[1] for row in xx]
    stationary.append(trapz(intval,t))
    #changed.append(stationaryval([row[1] for row in xx]))
    
    plt.plot(t, xx[:,1], label='I')
    #plt.plot(t, zz[:,1], label='Inormal')
    #plt.plot(t, plot1, label='Ifast')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    plt.legend()
    plt.show()
    return

def run1(a,b,c,d,e):
    for i in range(30):
        main(1,a+i*40,b,c,d,e)
    plt.scatter(entrypoint, stationary)
    #plt.scatter(entrypoint, changed, label='With pandemic')
    plt.xlabel('Time of Entry')
    plt.ylabel('Total infected')
    plt.title('Total infected vs Time of Entry')
    plt.legend()
    plt.show()
    return
run1(150,0.65,0.1,0.9,0.0001)

In [ ]:
!pip install ddeint
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pylab import *
from ddeint import ddeint

# Parameters (USING SWITCHED VALUES)


def gammanew(t,plot2):
    if t < entry:
        return gamma
    else:
        return max(0, gamma*(1 - a0*plot2[int(floor(t))]))

def modelslow(X,t,d1,plot2):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gammanew(t,plot2)*I, gammanew(t,plot2)*I - mu*Rd])

def modelslownormal(X,t,d1):
    S,I,R = X(t)
    Sd,Id,Rd = X(t-d1)
    return array([-beta*S*I/N + mu*Rd, beta*S*I/N - gamma*I, gamma*I - mu*Rd])

def model(Y,t,d):
    S,I,R = Y(t)
    Sd,Id,Rd = Y(t-d)
    return array([-ls*S*I/N + mu1*Rd, ls*S*I/N - g0*I, g0*I - mu1*Rd])


def stationaryval(values):
    count = 0
    prev = values[0]
    for i in range(len(values)-1):
        if count == 100:
            return prev
        if values[i+1] - prev <= 1:
            count += 1
        else:
            count = 0
        prev = values[i+1]
    return 0

stationary = []
changed = []
entrypoint = []

def main(marker,val1,val2,val3,val4,val5):
    marker = marker
    global entry
    entry = val1
    global eta
    eta = 1.4#val2
    global zeta
    zeta = 1.2#val3
    global factor
    factor = val4
    
    global beta
    beta = val2#0.25*0 + 0.5
    global gamma
    gamma = val3#1/15*0 + 0.1
    global mu
    mu = 0.097*0 + 0.11#0.1455
    global N
    N = 10000
    t_max = 1500
    global ls
    ls = eta*beta
    global g0
    g0 = zeta*gamma
    global a0
    a0 = val5#0.0001
    global mu1
    mu1 = (zeta*factor)*mu
    
    t_maxfast = t_max - entry
    
    S0, I0, R0 = N-1, 1, 0
    y0 = [S0, I0, R0]
    
    global stationary
    global changed
    global entrypoint

    # Time vector
    t = np.linspace(0, t_max, t_max + 1)
    tlist = list(t)
    t1 = np.linspace(0, t_maxfast, t_maxfast + 1)
    
    g = lambda t : array([S0,I0,R0])


    d = 10
    yy = ddeint(model,g,t1,fargs=(d,))

    add = np.zeros(entry)
    add1 = np.zeros(entry+10)
    plot = [row[1] for row in yy]
    plot = np.array(plot)
    plot1 = np.concatenate((add,plot))
    plot2 = np.concatenate((add1,plot))
    
    f = lambda t : array([S0,I0,R0])
    d1 = 10
    xx = ddeint(modelslow,f,t,fargs=(d1,plot2,))
    zz = ddeint(modelslownormal,f,t,fargs=(d1,))
    
    if marker == 1:
        entrypoint.append(val1)
    elif marker == 2:
        entrypoint.append(val2)
    elif marker == 3:
        entrypoint.append(val3)
    elif marker == 4:
        entrypoint.append(val5) 
    stationary.append(stationaryval([row[1] for row in zz]))
    changed.append(stationaryval([row[1] for row in xx]))
    
    #plt.plot(t, xx[:,1], label='I')
    #plt.plot(t, zz[:,1], label='Inormal')
    #plt.plot(t, plot1, label='Ifast')
    #plt.xlabel('Time')
    #plt.ylabel('Population')
    #plt.title(' Entry = '+str(val1)+' Eta = '+str(val2)+' Zeta = '+str(val3)+' Factor = '+str(val4)+' mu = '+str(val5))
    #plt.legend()
    #plt.show()
    return




def run1(a,b,c,d,e):
    for i in range(10):
        main(1,a+i*40,b,c,d,e)
    plt.scatter(entrypoint, stationary, label='Without Pandemic')
    plt.scatter(entrypoint, changed, label='With pandemic')
    plt.xlabel('Entry')
    plt.ylabel('Stationary values')
    plt.title('Entry vs Change in Stationary values')
    plt.legend()
    plt.show()
    return

def run2(a,b,c,d,e):
    for i in range(10):
        main(2,a,b+i*0.03,c,d,e)
    plt.scatter(entrypoint, stationary, label='Without Pandemic')
    plt.scatter(entrypoint, changed, label='With pandemic')
    plt.xlabel('Beta')
    plt.ylabel('Stationary values')
    plt.title('Beta vs Change in Stationary values')
    plt.legend()
    plt.show()
    return


def run3(a,b,c,d,e):
    for i in range(10):
        main(3,a,b,c+i*0.01,d,e)
    plt.scatter(entrypoint, stationary, label='Without Pandemic')
    plt.scatter(entrypoint, changed, label='With pandemic')
    plt.xlabel('Gamma')
    plt.ylabel('Stationary values')
    plt.title('Gamma vs Change in Stationary values')
    plt.legend()
    plt.show()
    return


def run4(a,b,c,d,e):
    for i in range(10):
        main(4,a,b,c,d,e+i*0.00002)
    entrypoint1 = [x*1000 for x in entrypoint]
    plt.scatter(entrypoint1, stationary, label='Without Pandemic')
    plt.scatter(entrypoint1, changed, label='With pandemic')
    plt.xlabel('a0 (x1e-3)')
    plt.ylabel('Steady state values')
    plt.title('a0 vs Steady state values')
    plt.legend()
    plt.show()
    return



run1(150,0.65,0.1,0.9,0.0001)

stationary.clear()
changed.clear()
entrypoint.clear()

run2(350,0.45,0.1,0.9,0.0001)

stationary.clear()
changed.clear()
entrypoint.clear()

run3(350,0.65,0.1,0.9,0.0001)

stationary.clear()
changed.clear()
entrypoint.clear()

run4(350,0.65,0.1,0.9,0.00006)

stationary.clear()
changed.clear()
entrypoint.clear()

